In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [5]:
train_data_gen = ImageDataGenerator(rescale = 1./255)
validation_data_gen = ImageDataGenerator(rescale = 1./255)

In [6]:
train_generator = train_data_gen.flow_from_directory('/kaggle/input/emotion-detection-fer/train', target_size=(48, 48), batch_size=64, color_mode='grayscale', class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [7]:
validation_generator = validation_data_gen.flow_from_directory('/kaggle/input/emotion-detection-fer/test',target_size=(48,48),batch_size=64, color_mode='grayscale',class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [8]:
emotion_model = Sequential()

In [9]:
emotion_model.add(Conv2D(32, kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
emotion_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation='relu'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
emotion_model_info=emotion_model.fit_generator(train_generator, steps_per_epoch=28709//64, epochs=50, validation_data=validation_generator, validation_steps=7178//64)

Epoch 1/50


/tmp/ipykernel_28/1514010088.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info=emotion_model.fit_generator(train_generator, steps_per_epoch=28709//64, epochs=50, validation_data=validation_generator, validation_steps=7178//64)


448/448 [==============================] - 98s 219ms/step - loss: 1.8586 - accuracy: 0.2492 - val_loss: 1.8123 - val_accuracy: 0.2473
Epoch 2/50
448/448 [==============================] - 33s 74ms/step - loss: 1.8544 - accuracy: 0.2512 - val_loss: 1.8341 - val_accuracy: 0.2475
Epoch 3/50
448/448 [==============================] - 33s 73ms/step - loss: 1.8243 - accuracy: 0.2515 - val_loss: 1.8191 - val_accuracy: 0.2476
Epoch 4/50
448/448 [==============================] - 33s 74ms/step - loss: 1.8155 - accuracy: 0.2579 - val_loss: 1.7847 - val_accuracy: 0.2758
Epoch 5/50
448/448 [==============================] - 35s 77ms/step - loss: 1.7911 - accuracy: 0.2720 - val_loss: 1.7641 - val_accuracy: 0.2909
Epoch 6/50
448/448 [==============================] - 34s 75ms/step - loss: 1.7598 - accuracy: 0.2953 - val_loss: 1.7449 - val_accuracy: 0.3089
Epoch 7/50
448/448 [==============================] - 33s 73ms/step - loss: 1.7425 - accuracy: 0.3107 - val_loss: 1.7044 - val_accuracy: 0.3221
Ep

In [12]:
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

emotion_model.save_weights("emotion_model.h5")

In [43]:
emotion_dict = {0:"Angry", 1:"Disgusted", 2:"Fearful", 3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprised"}

In [44]:
import numpy as np
from keras.models import model_from_json

In [45]:
json_file = open("/kaggle/working/emotion_model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

In [46]:
emotion_model.load_weights("emotion_model.h5")

In [105]:
import cv2

In [116]:
img = cv2.imread("/kaggle/input/emotion-detection-fer/train/sad/im1018.png", -1)
img = np.expand_dims(np.expand_dims(cv2.resize(img, (48, 48)), -1), 0)

In [117]:
emotion_prediction = emotion_model.predict(img)

1/1 [==============================] - 0s 19ms/step


In [118]:
maxindex = int(np.argmax(emotion_prediction))
print(emotion_dict[maxindex])

Sad
